# Quarter in-depth PI analysis

This report has been based on modified WANO PC approach. The purpose of this report is defining the units which need more support.

In [63]:
#install.packages('logging', repos='http://cran.us.r-project.org')

Analysing quarter

In [64]:
EndDate <- 201709

Read all the necessary data and function

In [65]:
source('shiny/data.r')

Criteria for FLR: the average two-year indicator is in the bottom decile in worlwide comparison

SP1, SP2 and SP5 - indicator is safety related.

Criteria for US7: 
- first filter: at least one scram occured during the last monitored quarter
- second filter: three or more scrams occured in the last 12 months

Criteria for FRI: calculated value is above the Defect Fuel Reference

In [66]:
#tail(r)

FLR: the average two-year indicator is in the bottom decile in worlwide comparison

In [67]:
flr_lim <- quantile(subset(r, IndicatorCode == 'FLR  ' & PeriodEndYrMn == EndDate & NumOfMonths == 24 & NonQualCode == ' ',ResultsValue)[[1]],probs=0.9)[[1]]
flr <- subset(r, IndicatorCode == 'FLR  ' & PeriodEndYrMn == EndDate & NumOfMonths == 24 & NonQualCode == ' ' & ResultsValue>=flr_lim,LocId)[[1]]
flr <- nameByID(flr)

Units which is above the safety system criteria (0.02 for SP1-2, 0.025 for SP5) for 3-yrs data window

In [68]:
sp12 <- nameByID(subset(r, IndicatorCode %in% c('SP1  ','SP2  ') & PeriodEndYrMn == EndDate & NumOfMonths == 36 & NonQualCode == ' ' & ResultsValue >=0.02,LocId)[[1]])
sp5 <- nameByID(subset(r, IndicatorCode == 'SP5  ' & PeriodEndYrMn == EndDate & NumOfMonths == 36 & NonQualCode == ' ' & ResultsValue >=0.025,LocId)[[1]])

Units which has at least one scram occured during the last monitored quarter

In [69]:
#tail(data)
scram <-nameByID(unique(subset(data,YrMn %in% c(EndDate-2, EndDate-1 ,EndDate) & ElementCode %in% c('C1   ','C3   ') & ElementValue>0, SourceId))[[1]])

Units which has three or more scrams for the last 12 months

In [70]:
scrams3 <- nameByID(unique(subset(data,YrMn > (EndDate-100) & ElementCode %in% c('C1   ','C3   ') & ElementValue>=3, SourceId))[[1]])

FRI: calculated value is above the Defect Fuel Reference

In [71]:
u <- uByType()
fri2 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=300 & LocId %in% u$rType[[2]],LocId)[[1]])
fri45 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=5e-4 & LocId %in% c(u$rType[[4]],u$rType[[5]]),LocId)[[1]])
fri1 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=1e-3 & LocId %in% u$rType[[1]],LocId)[[1]])
fri3 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=1e-3 & LocId %in% u$rType[[3]],LocId)[[1]])
fri <- c(fri1,fri2,fri3,fri45)

Note: FRI criteria for AGR and LWCGR are based on Index calculation

The whole list of units which have some issues

In [72]:
onFocus <- sort(unique(c(flr,sp12,sp5,scram,scrams3,fri)))

In [74]:
df <- data.frame(onFocus)
flrCol <- c()
sspiCol <-c()
S <- c()
S3 <- c()
FRI <- c()

for (i in df$onFocus){
    if (i %in% flr) flrCol <- c(flrCol,'X') else flrCol <- c(flrCol,'')
    if (i %in% c(sp12,sp5)) sspiCol <- c(sspiCol,'X') else sspiCol <- c(sspiCol,'')
    if (i %in% scram) S <- c(S,'X') else S <- c(S,'')
    if (i %in% scrams3) S3 <- c(S3,'X') else S3 <- c(S3,'')
    if (i %in% fri) FRI <- c(FRI,'X') else FRI <- c(FRI,'')    
}
df$FLR <- flrCol
df$SSPI <- sspiCol  
df$Scram <- S
df$Scrams <- S3
df$FRI <- FRI
df
write.csv(df,'onFocus.csv')

onFocus,FLR,SSPI,Scram,Scrams,FRI
ANO,,X,,,
ANO Unit 2,X,X,,,
Atucha 1,,,,X,X
Atucha 2,X,,,,X
Blayais 4,,,X,,
Borssele 1,,,X,,
Bruce 1,,,X,,
Bruce 7,,X,,,
Brunswick Unit 1,,,,,X
Bugey 2,,,X,,


The next stages will be:
- check and print the related comments (if any);
- search the related events in the OE DB
- provide reccomendations for PR team
- add some text summary
- remove code